In [12]:
import nltk
import math

In [18]:
122<math.inf

True

In [3]:
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [39]:
from nltk.corpus import wordnet as wn

wn.synsets('primary_election')[0].name()


'primary.n.01'

In [5]:
gloss = word_tokenize(wn.synset('state.v.01').definition())

words_in_gloss = [word for word in gloss if not word in stopwords.words()]


'express in words'

In [7]:
wn.synsets("state")

[Synset('state.n.01'),
 Synset('state.n.02'),
 Synset('state.n.03'),
 Synset('state.n.04'),
 Synset('state_of_matter.n.01'),
 Synset('state.n.06'),
 Synset('country.n.02'),
 Synset('department_of_state.n.01'),
 Synset('state.v.01'),
 Synset('submit.v.02'),
 Synset('express.v.04')]

In [63]:
nltk.stop_words()

AttributeError: module 'nltk' has no attribute 'stop_words'

In [64]:
example_sent=wn.synset('bank.n.03').definition()

In [70]:
stop_words = set(stopwords.words('english'))
 
word_tokens = word_tokenize(example_sent)
 
filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
 
filtered_sentence = []
 
for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)
 
print(word_tokens)
print(filtered_sentence)

['a', 'long', 'ridge', 'or', 'pile']
['long', 'ridge', 'pile']


In [53]:
string=""
for i in wn.synset('bank.n.02').examples():
    string += " "+i
string.strip()

'he cashed a check at the bank that bank holds the mortgage on my home'

In [45]:
[i.lemma_names() for i in wn.synsets('bank')]

[['bank'],
 ['depository_financial_institution',
  'bank',
  'banking_concern',
  'banking_company'],
 ['bank'],
 ['bank'],
 ['bank'],
 ['bank'],
 ['bank', 'cant', 'camber'],
 ['savings_bank', 'coin_bank', 'money_box', 'bank'],
 ['bank', 'bank_building'],
 ['bank'],
 ['bank'],
 ['bank'],
 ['bank'],
 ['bank'],
 ['bank'],
 ['deposit', 'bank'],
 ['bank'],
 ['trust', 'swear', 'rely', 'bank']]

In [17]:
wn.lemma_from_key(wn.synset('probe.n.01').offset())

NameError: name 'wn' is not defined

In [10]:
wn.synsets('probe')[0].definition()

'an inquiry into unfamiliar or questionable activities'

In [47]:
dir(wn.lemma('eat.v.03.eat'))

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_frame_ids',
 '_frame_strings',
 '_hypernyms',
 '_instance_hypernyms',
 '_key',
 '_lang',
 '_lex_id',
 '_lexname_index',
 '_name',
 '_related',
 '_synset',
 '_syntactic_marker',
 '_wordnet_corpus_reader',
 'also_sees',
 'antonyms',
 'attributes',
 'causes',
 'count',
 'derivationally_related_forms',
 'entailments',
 'frame_ids',
 'frame_strings',
 'hypernyms',
 'hyponyms',
 'in_region_domains',
 'in_topic_domains',
 'in_usage_domains',
 'instance_hypernyms',
 'instance_hyponyms',
 'key',
 'lang',
 'member_holonyms',
 'member_meronyms',
 'name',
 'part_holonyms',
 'part_meronyms',
 'pertainyms',
 'region_dom

In [19]:

import numpy as np
import itertools
import matplotlib
import nltk
import random
import math
import matplotlib.pyplot as plt
import statistics
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from nltk.tree import Tree
from nltk.corpus import semcor


# Getting the brown corpus
print("Downloading files from NLTK please wait...")
nltk.download('all', quiet=True)
print("NLTK files downloaded!")



# Task: Tag words with synset ID
# Three types of tags in semcor
# 1) No tag
# 2) Tagged with a lemma of synset: A lemma is a data structure which contains synset along with its synset_id, hypernymy, hyponymy,... 
# 3) Tagged with lemma but the synset ID doesn't exist in wordnet
#    One such example is the 7th sentence in the semcor corpus
#    The grand jury commented on a number of other topics, among them the Atlanta and Fulton County purchasing departments which it said are 
#    well operated and follow generally accepted practices which inure to the best interest of both governments.
#                                       ^______^  "accepted" is tagged as "accepted.s.00" lemma and this doesn't have a synsetID in wordnet
#    All the three cases are handled by the code

# Each sentence has a pair of phrase and (representation of)synset
# Each Phrase has one or more words(e.g. "primary election" in first sentence of corpora)
# Each synset is respresented as a Tree e.g. for "primary election"
# Lemma('primary.n.01.primary_election')
#                 /\
#                /  \
#               /    \
#         primary   election
# This example will be used later to explain code
print("Preprocessing the semcor corpus...")
synCorpus=[]
current_index=0
for taggedSents in semcor.tagged_sents(tag='sem'): # for each sentence fetch the (phrase,synsetTree)
    synCorpus.append([])                           # create a list designating current sentence which will contain (phrase,synsetID)
    for phrase in taggedSents:
        if type(phrase)==list:                     # The phrase with no tag is a list in semcor [Note: CASE 1]
            #if its a list fetch the word and set -1 as synset ID which indicates no synset TAG
            # synCorpus[current_index].append((phrase[0],-1))  # a tuple is formed
            pass
        else: # if synset is a Tree [Note Case 2 and 3]
            # Here when theres a synset not in wordnet, theres no synset id[Case 3], to handle this
            # except block is used
            # The try except will get the synset ID and in case no synsetID -2 is given to such a case
            try: 
                # handle Case 2
                synCorpus[current_index].append((phrase.label().name(),phrase.label().synset().name())) 
            except:
                # handle Case 3
                #synTag_for_corpus=-2 # -2 indicates no synset tags for the specific POS but available for some POS
                pass
    current_index+=1 # Keeps tract of the sentence index
no_sents=current_index #Saved total number of sentences
# The reason for storing as tuples is because the brown corpus stored it in a same way and this way we have to make minimal changed to HMM-Viterbi POS tag code if any :)
print("Semcor preprocessing finished")


NLTK files downloaded!
Preprocessing the semcor corpus...
Semcor preprocessing finished


In [26]:
synCorpus[0]

[('group', 'group.n.01'),
 ('say', 'state.v.01'),
 ('Friday', 'friday.n.01'),
 ('investigation', 'probe.n.01'),
 ('Atlanta', 'atlanta.n.01'),
 ('recent', 'late.s.03'),
 ('primary_election', 'primary.n.01'),
 ('produce', 'produce.v.04'),
 ('evidence', 'evidence.n.01'),
 ('irregularity', 'abnormality.n.04'),
 ('take_place', 'happen.v.01')]

In [50]:
WFS=[]
current=0
for sent in synCorpus:
    WFS.append([])
    for (word,tag) in sent:
          WFS[current].append((word,wn.synsets(word)[0].name()))
    current+=1

In [55]:
WFS[2]

[('September', 'september.n.01'),
 ('October', 'october.n.01'),
 ('term', 'term.n.01'),
 ('jury', 'jury.n.01'),
 ('charge', 'charge.n.01'),
 ('location', 'location.n.01'),
 ('person', 'person.n.01'),
 ('investigate', 'investigate.v.01'),
 ('report', 'report.n.01'),
 ('possible', 'possible.n.01'),
 ('irregularity', 'abnormality.n.04'),
 ('hard-fought', 'hard-fought.s.01'),
 ('primary', 'primary.n.01'),
 ('win', 'win.n.01'),
 ('person', 'person.n.01')]

In [80]:
from nltk.corpus import wordnet as wn
total_count=0
correct_count=0
current_sent=0
for sent in synCorpus:
    current_word=0
    for (word,tag) in sent:
        if tag==WFS[current_sent][current_word][1]:
            correct_count+=1
        total_count+=1
        current_word+=1
    current_sent+=1

In [84]:
correct_count/total_count

0.4868901190836434

In [86]:
mfs_dict = {}
only_words=[]
original_ids=[]
for sent in synCorpus:
    sentence=[]
    sent_id=[]
    for (word,tag) in sent:
        word=word.lower()
        try:
            try:
                mfs_dict[word][tag]+=1
            except:
                mfs_dict[word][tag]=1
        except:
            mfs_dict[word]={tag:1}
        sentence.append(word)
        sent_id.append(tag)
    only_words.append(sentence)
    original_ids.append(sent_id)

In [88]:
mfs_baseline_pred=[]
for sent in only_words:
    baseline_pred_sent=[]
    for word in sent:
        sense_dict=mfs_dict[word]
        pred_sense=max(zip(sense_dict.values(),sense_dict.keys()))[1]
        baseline_pred_sent.append(pred_sense)
    mfs_baseline_pred.append(baseline_pred_sent)

In [96]:
total_count_MFS=0
correct_count_MFS=0
current_sentence_MFS=0
for sentence in mfs_baseline_pred:
    current_word_MFS=0
    for tag in sentence:
        if tag==original_ids[current_sentence_MFS][current_word_MFS] and tag!='group.n.01':
            correct_count_MFS+=1
        total_count_MFS+=1
        current_word_MFS+=1
    current_sentence_MFS+=1

In [97]:
correct_count_MFS/total_count_MFS

0.6910811869203795

In [27]:
  train_word_tag = {}
  for sent in synCorpus:
    for (word,tag) in sent:
      try:
        try:
          train_word_tag[word][tag]+=1
        except:
          train_word_tag[word][tag]=1
      except:
          train_word_tag[word]={tag:1}

In [28]:
train_word_tag

{'group': {'group.n.01': 1328,
  'group.n.02': 1,
  'group.v.01': 3,
  'group.v.02': 1},
 'say': {'state.v.01': 1684,
  'allege.v.01': 216,
  'order.v.01': 8,
  'suppose.v.01': 27,
  'pronounce.v.01': 4,
  'say.v.08': 1,
  'read.v.02': 8,
  'say.v.07': 2},
 'Friday': {'friday.n.01': 18},
 'investigation': {'probe.n.01': 15, 'investigation.n.02': 6},
 'Atlanta': {'atlanta.n.01': 7, 'atlanta.n.02': 1},
 'recent': {'late.s.03': 9, 'recent.s.01': 36},
 'primary_election': {'primary.n.01': 1},
 'produce': {'produce.v.04': 6,
  'produce.v.01': 67,
  'produce.v.06': 6,
  'produce.v.03': 26,
  'grow.v.07': 6,
  'produce.v.02': 37,
  'produce.n.01': 2},
 'evidence': {'evidence.n.01': 53,
  'evidence.n.02': 21,
  'attest.v.01': 10,
  'evidence.n.03': 7,
  'testify.v.02': 3,
  'tell.v.07': 1},
 'irregularity': {'abnormality.n.04': 3,
  'irregularity.n.03': 1,
  'irregularity.n.02': 2},
 'take_place': {'happen.v.01': 53},
 'jury': {'jury.n.01': 21, 'jury.n.02': 2},
 'far': {'far.r.02': 27,
  'far.